# eCommerce Example

This notebook details a time series analysis and forecasting application performed with scalecast using the eCommerce dataset. The notebook starts with an exploratory data analysis; moves to time series decomposition; forecasts with an exponential smoothing model; an ARIMA model; a multiple linear regression model; moves to automated forecasting with scikit-learn models, Facebook Prophet, and LinkedIn Greykite/Silverkite; explores TensorFlow recurrent neural nets; and finally finishes with combination modeling.

The utilized dataset is available on kaggle: https://www.kaggle.com/carrie1/ecommerce-data/

[Library Imports](#Library-Imports)  
[Exploratory Data Analysis](#Exploratory-Data-Analysis)  
[Forecast with scalecast](#Forecast-with-Scalecast)  

  - [Holt-Winters Exponential Smoothing](#HWES)  
  - [ARIMA](#ARIMA)  
  - [MLR](#MLR)  
  - [Elasticnet and Auto-Forecasting](#Elasticnet-and-Auto-Forecasting)  
  - [Auto-Forecasting the Scikit-learn Models](#Auto-Forecasting-the-Scikit-learn-Models)  
  - [Prophet and Silverkite](#Prophet-and-Silverkite)  
  - [TensorFlow Recurrent Neural Nets](#TensorFlow-Recurrent-Neural-Nets)  
    - [SimpleRNN](#SimpleRNN)  
    - [LSTM](#LSTM)  
  - [Combination Modeling](#Combination-Modeling)  

[Export Results](#Export-Results)  

## Library Imports
[Back to top](#eCommerce-Example)  
First, let's import the libraries and read the data. Some data preprocessing in pandas will be necessary before calling scalecast.

In [ ]:
import pandas as pd
import numpy as np
import datetime
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from scalecast.Forecaster import Forecaster
from scalecast import GridGenerator
from scalecast.notebook import tune_test_forecast

In [ ]:
data = pd.read_csv('eCommerce.csv',parse_dates=['InvoiceDate'], encoding = 'unicode_escape')

In [ ]:
sns.set(rc={'figure.figsize':(15,7)})

## Exploratory Data Analysis

[Back to top](#eCommerce-Example)  
Let's view the data's first five rows.

In [ ]:
data.head()

Now, let's view the data's dimensions.

In [ ]:
data.shape

Let's see how much time these 550,000 observations span.

In [ ]:
print('first date in data:',data['InvoiceDate'].min())
print('last date in data:',data['InvoiceDate'].max())

In spite of there being over half a million rows, there is only about a year's worth of data to analyze. Before proceding, we should decide a datetime frequency to aggregate the data to, otherwise we will be trying to forecast with an incosistent interval of time between observations. This decision depends on what question we are trying to answer. For this example, let's try answering the question of if we can accurately predict daily gross sales revenues in the United Kingdom over 30 days. This means we will be removing sales that were negative in value (probably representing returns), creating a "Sales" column by multiplying quantity by price, and aggregating the entire dataframe to the daily level. Then, we subset to `country=='United Kingdom'` and fill any days that show no sales with 0.

In [ ]:
# drop negative sales quantities
data = data.loc[(data['Quantity'] > 0) & (data['Country'] == 'United Kingdom')]
# create the Sales column
data['Sales'] = data['Quantity']*data['UnitPrice']
# aggregate the dataframe to the daily level
dt_aggr = 'D'
data['DateTime'] = data['InvoiceDate'].dt.floor(dt_aggr)
tbl = data.groupby('DateTime')['Sales'].sum().reset_index()
# view first 5 rows
tbl.head()

In [ ]:
tbl.shape

In [ ]:
print('first date in data:',tbl['DateTime'].min())
print('last date in data:',tbl['DateTime'].max())

It is possible that after making this aggregation to the daily level, some date observations in the range are missing. Some forecasting libraries will process missing data for you automatically, but because scalecast mixes so many model concepts, it is necessary to have every possible date in a given range represented. If we run the code below, we will limit the dataframe to only days and any missing dates will have their sales filled with 0.

In [ ]:
all_dates = pd.DataFrame({'DateTime':pd.date_range(start=tbl['DateTime'].min(),end=tbl['DateTime'].max(),freq=dt_aggr)})
full_data = all_dates.merge(tbl,on='DateTime',how='left').sort_values(['DateTime']).fillna(0)
full_data.head()

Let's see how that looks plotted.

In [ ]:
full_data.plot(x='DateTime',y='Sales',title='eCommerce Sales Original')
plt.show()

We notice the last observation in the dataframe is an outlier. Let's look at it more closely.

In [ ]:
full_data.sort_values(['Sales'],ascending=False).head(1)

There are different ways to handle outliers in time series, but we will try ignoring it to see how well our models can predict it.

The last preprocessing function we want to perform is removing the first month or so of observations because so many of them are 0 or close-to-0 in value. Starting January 4, 2011, we see a more normal pattern.

In [ ]:
full_data = full_data.loc[full_data['DateTime'] >= datetime.datetime(2011,1,4)]
full_data.plot(x='DateTime',y='Sales',title='eCommerce Sales Starting Jan 4')
plt.show()

Much better. In the real world, we might want to know more about that outlier -- why it exists, the best way to mitigate it, etc. This example is more interested in producing forecasts and showing the scalecast process, so we are going to do a very simple mitigation strategy -- cap the outlier at the next highest value.

In [ ]:
full_data.iloc[-1,1] = full_data.iloc[:-1,:]['Sales'].max()

Let's now try to get a better idea of how the data is distributed.

In [ ]:
full_data.describe()

In [ ]:
full_data['Sales'].hist(bins=25)
plt.show()

The remaining data looks somewhat normally distributed with a slight right skew.

## Forecast with Scalecast
[Back to top](#eCommerce-Example)  
To load the object, we call the Forecaster() function with the `y` and `current_dates` parameters specified. If we hadn't already dropped the first observations of the data before calling the object, we could have done it by using the `keep_smaller_history()` function as shown below. We can then plot the values that we will be using for forecasting.

In [ ]:
f = Forecaster(y=full_data['Sales'],current_dates=full_data['DateTime'])
f

The first thing you should do after initializing the object is set its test length. What that length is is up to you. The longer the length, the more confident you can be about your reported error/accuracy metrics. The library requires a test length of at least 1. Let's set our test length to be the same size as our forecast length: 30 days.

In [ ]:
f.set_test_length(30)

Before beginning the forecasting process, we should get a better idea of the signals within the time series itself. Using ACF, PACF, and Periodogram plots, we can observe how the series is auto-correlated. We leave the test set out of all visualizations (`train_only=True`) to not leak data when making decisions about which signals exist in the data.

In [ ]:
f.plot_acf(train_only=True,lags=30)
plt.show()

In [ ]:
f.plot_pacf(train_only=True,lags=30)
plt.show()

In [ ]:
a, b = f.plot_periodogram(diffy=True,train_only=True)
plt.semilogy(a, b)
plt.show()

In [ ]:
f.seasonal_decompose(train_only=True).plot()
plt.show()

From these graphs, we get a sense that the data has a seasonal pattern over seven periods, or one week. The data appears to be significantly autocorrelated back 1, 6, 7, and 13, and 14 periods, possibly more. This is good information to use when deciding how to specify the forecasts. 

Now, let's test the data's stationarity using the Augmented Dickey-Fuller test. The null hypothesis of this test is that the data is not stationary. To return the p-val and critical value from this test, pass `full_res=True` as an argument. The way we have the test specified, it will simply print out the implications from the test.

In [ ]:
isstationary = f.adf_test(quiet=False)

Since our test implies non-stationarity, let's view all these plots again, but pass differenced data into them by using the `diffy=1` argument.

In [ ]:
f.plot_acf(diffy=1,train_only=True,lags=30)
plt.show()

In [ ]:
f.plot_pacf(diffy=1,train_only=True,lags=30)
plt.show()

In [ ]:
f.seasonal_decompose(diffy=1,train_only=True).plot()
plt.show()

This doesn't give us a very different view of the data, but reinforces that there is strong autocorrelation in our dataset. Some of that can be controlled by using the series' first difference.

You can see below all functions available to plot the information in the object:

In [ ]:
print(*f.get_funcs('plotter'),sep='\n')

All setter functions:

In [ ]:
print(*f.get_funcs('setter'),sep='\n')

All getter functions:

In [ ]:
print(*f.get_funcs('getter'),sep='\n')

### HWES
[Back to top](#eCommerce-Example)  
Let's run a couple of exponential smoothing forecasts using Holt-Winters Exponential Smoothing from StatsModels. Most exponential smoothing is considered a somewhat simple way to forecast time series. They are models that smooth out recent trends and predict them to the future. The added benefit of using Holt-Winters is that adding extra parameters to this basic idea, such as seasonality, is possible. We will try one HWES model with seasonality and one without.

Before running any forecast, we need to generate a forecast period. We are attempting to predict 30 days into the future.

In [ ]:
f.generate_future_dates(30)

To run an HWES model, we first set the estimator to hwes, then call the `manual_forecast()` function. We can run as many hwes models as we like and differentiate them by using the `call_me` argument. By default, the model will be called whatever the estimator is ("hwes" is "hwes", "arima" is "arima", etc.)

In [ ]:
f.set_estimator('hwes')
f.manual_forecast(trend='add')
f.save_summary_stats()

f.manual_forecast(trend='add',seasonal='add',call_me='hwes_seasonal')
f.save_summary_stats()

We can view the results of the model by plotting the test-set predictions with the actual test-set observations, setting ci=True to show 95% confidence intervals. Any confidence level can be set by using `f.setcilevel(...)`. The intervals are evaluated with bootstrapping on the fitted values, and could be different than what is returned from the underlying statsmodels function. This is to make all forecast comparisons consistent across all model types. If the forecast shows tight confidence intervals but is less accurate on the test set, this is a good indication that your model is overfit.

In [ ]:
f.plot_test_set(ci=True)

We can see the models' performance over the 30-day forecast horizon as well:

In [ ]:
f.plot(print_attr=['LevelTestSetRMSE'],ci=True)

You may have noticed we called a function `save_summary_stats()` after running each ARIMA model. Most models allow feature information to be saved from them using `save_feature_importance()` or `save_summary_stats()`. Some models don't allow either. These functions should should be called before running a new model because they automatically save that information for the last model run only. As a rule of thumb, saving summary stats is much less computationally expensive than saving feature importance, which will be explored later.

We can see the summary statistics for the better-peforming HWES model exporting the saved summary stats to a dataframe.

In [ ]:
f.export_summary_stats(model='hwes_seasonal')

### ARIMA
[Back to top](#eCommerce-Example)  
Now, let's run another common, albeit slightly more advanced time-series model: ARIMA. This model uses the series' own history, errors, and stationarity to forecast. Using the output from the plots above, as well as the results from the ADF test, we can specify a 1,1,0 x 1,1,0, as well as a 1,1,1 x 0,1,1, ordered model. The seasonal period will be 7 periods--one week.

In [ ]:
f.set_estimator('arima')
f.manual_forecast(order=(1,1,0),seasonal_order=(1,1,0,7))
f.save_summary_stats()

f.manual_forecast(order=(1,1,1),seasonal_order=(0,1,1,7),call_me='arima_ma_terms')
f.save_summary_stats()

We can view the results of the model by plotting the test-set predictions with the actual test-set observations.

In [ ]:
f.plot_test_set(models=['arima','arima_ma_terms'],ci=True)

Those ARIMA model with MA terms appears to capture the daily trend fairly well. Not so much for the other ARIMA model. Let's see how they look into future periods compared to the HWES models previously evaluated.

In [ ]:
f.plot(print_attr=['LevelTestSetRMSE'],ci=True)

The ARIMA model without MA terms, as well as the hwes model without seasonality, is the worst performing models we have evaluated, using the test-set RMSE as our comparison metric. Let's delete both of them from memory.

In [ ]:
f.pop('arima','hwes')

We can see the summary statistics for the remaining ARIMA model by exporting the saved summary stats to a dataframe, just like we did for the HWES model.

In [ ]:
f.export_summary_stats(model='arima_ma_terms')

### MLR
[Back to top](#eCommerce-Example)  
We can use many other models through scalecast. One of the most basic of these is Multiple Linear Regression. Unlike ARIMA, we don't specify orders on this model, but we can add similar regressors, including autoregressive and seasonal terms, as well as a time trend. The MLR works very similarly to the ARIMA model in that it tries to find a linear relationship between all these components and the future. Unlike ARIMA, it makes the assumption that the errors in each time period are independent of one another. This assumption may be spurious, but the MLR has proven to make accurate predictions on real-world data.

First, let's begin with autoregressive terms, which are lags of the dependent variable values. We can add 28 lags to make sure we are capturing all statistically signficant terms, according to the ACF and PACF plots we viewed earlier.

In [ ]:
f.add_ar_terms(28) # 1-28 lags

To account for stationarity, which is done by setting the middle term to 1 in ARIMA, we have to difference our data before modeling with linear regression.

In [ ]:
f.diff()

We can confirm the first-differenced data's (probable) stationarity with another ADF test.

In [ ]:
isstationary = f.adf_test(quiet=False)

Let's plot its first difference.

In [ ]:
f.plot(models=None)

Let's now add additional seasonality to the model regressors. The main seasonality that we have been able to confirm is weekly, so we can add the 'dayofweek' regressor to the object. We have three options: we can use a sin/cos transformation that accounts for regular fluctuations in the data in a cyclical form; we can add the data as 6-7 dummy variables using `dummy=True` and specifying the `drop_first` parameter; or we can just use the raw 0-6 numerical output, which is the default (a decision tree model may handle this last kind of regressor better than a linear model). Any decision we make in this regard has its pros and cons. For this example, we will use the dummy transformation. Also, to add complexity that the ARIMA couldn't caputre, we add weekly, monthly, and quarterly seasonality with the sin/cos transformation.

Other seasonal regressors are available and can be specified in the same way, including 'day', 'hour', 'minute' and more.

In [ ]:
f.add_seasonal_regressors('dayofweek',raw=False,dummy=True,drop_first=True)
f.add_seasonal_regressors('week','month','quarter',raw=False,sincos=True)

Let's also add a time trend.

In [ ]:
f.add_time_trend()

All options for adding regressors are:

In [ ]:
print(*f.get_funcs('adder'),sep='\n')

Let's see what calling our object now that we have evaluated some models looks like.

In [ ]:
f

When adding these and other kinds of regressors, it is possible to change the names of some of them. This can come in handy to reference later. If any regressors begin with uppercase "AR", the forecasting mechanisms in most of the models will assume such terms are autoregressive in nature and those terms are handled differently. So, be careful when naming variables.

Let's move to modeling with a linear model. Any arguments that the linear model from the scikit-learn library accepts can also be accepted here. In addition, the following arguments are available for all sklearn models:
- `Xvars` (arguments include "all", None, and list-like objects) -- default is always None, but for models that require regressors, this is treated the same as "all"
- `normalizer` (arguments are None, "minmax", "normalize", "pt", and "scale") -- default is always "minmax"
- `call_me` -- does not affect the model's evaluation at all, just names the model for reference later in case we want to run multiple models of the same class

In [ ]:
f.set_estimator('mlr')
f.manual_forecast(normalizer=None,Xvars=None)

The way the forecasting mechanism works for all Scikit-learn models, including MLRk (when AR terms are involved), is by making a prediction one step into the future then filling in those predictions to create new AR terms, until the entire forecast interval has been predicted. This is true for testing and forecasting, but validating is non-dynamic by default. Both validating and testing can be either dynamic or non-dynamic; this will be explored later. With large test, validation, or forecast intervals, the forecasting may slow down considerably if everything is kept dynamic. However, if AR terms have *not* been added to the regressors, forecasting times are similar to any non-time-series prediction application.

Like the ARIMA model, we can see its performance on the test set. 

In [ ]:
f.plot_test_set(models='mlr',ci=True)

Since the other models were run on level data and the MLR was run on differenced data, to compare them, the plots of the test set and forecasts will revert to level automatically, but `level=True` is available as an argument if all models were run on differenced data but you want to see their performance on the level test set.

In [ ]:
f.plot_test_set()

Confidence intervals are no longer available since models were run at different levels.

Let's now plot both models' forecasted values on level data. Let's order the way these models are displayed based on their test-set performance as well.

In [ ]:
f.plot(print_attr=['LevelTestSetRMSE'],order_by='LevelTestSetRMSE')

We see that the MLR model is the worst-performing so far.

With scikit-learn models, such as MLR, permutation feature importance information from the [eli5](https://eli5.readthedocs.io/en/latest/blackbox/permutation_importance.html) package can be saved and exported to a dataframe, analogous to saving summary stats from an ARIMA model. Each weight is the average decrease in accuracy when that particular variable is substituted for random values over 10 iterations, so the variables that cause the largest decreases are considered the most important. The output of the exported data is therefore ranked in terms of which variables are most-to-least important, according to this methodology.

In [ ]:
f.save_feature_importance()

In [ ]:
f.export_feature_importance(model='mlr')

At first glance, it looks as though almost all added regressors were at least somewhat useful to the MLR model, but we can explore regularization now to further refine our forecasting approach.

### Elasticnet and Auto-Forecasting

[Back to top](#eCommerce-Example)  
To optimize the hyperparameters of and auto-forecast with models in scalecast, we use a grid-search approach on a validation set of data--a period of time before the test set. The grids are completely customizable, but standard template grids are available by calling the function below:

In [ ]:
GridGenerator.get_example_grids()

`GridGenerator.get_empty_grids()` is also available. We could call this then open the created file (Grids.py) and fill in the empty dictionary with hyperparameter values that we want to use. The example grids can usually be used for adequate performance, but contributions to improving the default values are welcome.

These grids are saved to your working directory as Grids.py. Their structure is that of `Dict[str:list-like]` and scalecast automatically knows how to look for them. You can also pass your own grid manually by using `ingest_grid()` and passing a `str` value, which corresponds to a namespace of a grid in the Grids.py file, or a grid of `dict` type. By default, the code below will ingest the grid named elasticnet from the Grids.py file in this same directory.

You can also create really big grids and limit them randomly by calling `limit_grid_size(n:int|float)`. If `int`, must be >0 to denote the number of combinations in the grid to keep. If `float`, must be between 0 and 1 to represent a portion of the grid's original size to randomly keep. `random_seed` is available as an argument in this method and is `None` by default.

In [ ]:
f.set_validation_length(15)
f.set_estimator('elasticnet')
# unlike testing, tuning is non-dynamic by default, but this can be changed by passing dynamic_tuning=True as an argument below
f.tune() # automatically imports the elasticnet grid from Grids.py
# unlike tuning, testing is dynamic by default, but this can be changed by passing dynamic_testing=False as an argument below
f.auto_forecast()
f.save_feature_importance()

In [ ]:
f.plot_test_set(models=['mlr','elasticnet'],ci=True)

Regularization really took the seasonal pattern out of this model and reverted the prediction to nearly a straight line. Let's compare all model forecasts as we did before.

In [ ]:
f.plot(print_attr=['LevelTestSetRMSE'],order_by='LevelTestSetRMSE')

In this instance, the regularization performed by the Elasticnet model resulted in worse performance than the MLR. We can see what hyperparameter values were selected from the tuning process by exporting the evaluated validation grid to a dataframe.

In [ ]:
f.export_validation_grid(model='elasticnet').sort_values(['metric_value']).head(15)

Since this model has a 0 value for its l1_ratio, it is functionally identical to a ridge model. The selected alpha value was 2.0, meaning its coefficient values were enhanced. Like the MLR, we can export Elasticnet feature importance:

In [ ]:
f.export_feature_importance(model='elasticnet')

The elasticnet may have been over-parameterized as it now finds some features as slightly harmful to the model, although those low of values could be due to chance.

### Auto-Forecasting the Scikit-learn Models

[Back to top](#eCommerce-Example)  
In the same way we automatically tuned and forecasted with an elasticnet model, we can choose many models to forecast with using `notebook.tune_test_forecast()`. We will be tuning and forecasting with all available scikit-learn models. Below is a list of all models available, whether they are from scikit-learn, and whether they can be tuned.

In [ ]:
from scalecast.Forecaster import _estimators_, _can_be_tuned_, _sklearn_estimators_

for m in _estimators_:
    print(f'{m}; can be tuned: {m in _can_be_tuned_}; from scikit-learn: {m in _sklearn_estimators_}')

Let's call the function and the scikit-learn models only and save all feature info so that we can export that information later. To run the next function, note the following dependencies:

- `pip install tqdm`
- `pip install ipython`
- `pip install ipywidgets`
- `jupyter nbextension enable --py widgetsnbextension`
- if using Jupyter Lab: `jupyter labextension install @jupyter-widgets/jupyterlab-manager`

In [ ]:
tune_test_forecast(f,
                   _sklearn_estimators_,
                   dynamic_tuning=False,
                   dynamic_testing=True,
                   feature_importance=True) # dynamic_tuning = False and dynamic_testing = True are defaults in this function

For the amount of models that were evaluated and considering that dynamic testing was utilized, this evaluation is fairly quick! See the results on the test set.

In [ ]:
f.plot_test_set(order_by='LevelTestSetRMSE',models=_sklearn_estimators_,ci=True)

All of these models appear to have performed well on test-set data. Using a variety of models and tuning them carefully, a couple (or more) usually emerge as appearing reasonable to implement. Let's now see the forecasts and level test-set RMSE from all these models.

In [ ]:
f.plot(models=_sklearn_estimators_,order_by='LevelTestSetRMSE',print_attr=['LevelTestSetRMSE'],ci=True)

Surprisingly perhaps, nothing outperformed the MLR model. And the MLR wasn't as good as the ARIMA or seasonal HWES model. Sometimes the simplest model is the best model. Let's see the forecasted plots for each model, but to avoid clutter, we focus on only the models that came out further ahead than the others. We pass the `models='top_5'` argument to this function and rerun. Note, now that the HWES model will not be displayed (which was run on level data), we will have to set `level=True` to view the forecasts at level.

In [ ]:
f.plot(models='top_5',order_by='LevelTestSetRMSE',print_attr=['LevelTestSetRMSE'],level=True)

These various patterns are interesting, and one has to believe the last value being high signficantly influenced the resulting trends of the lightGBM and MLP models.

### Prophet and Silverkite

[Back to top](#eCommerce-Example)  
In addition to the scikit-learn and statsmodels forecasting models we have already explored, two other forecasting models from popular libraries are available for use: Facebook [Prophet](https://facebook.github.io/prophet/) and LinkedIn Silverkite (from the [greykite](https://engineering.linkedin.com/blog/2021/greykite--a-flexible--intuitive--and-fast-forecasting-library) package). Prophet using a Bayesian regression approach with changepoints; Silverkite uses a linear model with regularization, also with changepoints. Calling them and using them are as easy as any other forecasting model in this package. The `Xvars` argument is available in both models--you can also tune them using the grid-search method. But, since they select a lot of their own regressors and optimize themselves by default, it is sometimes time-saving to manually forecast them with default parameters. We can also run them on un-differenced data and let them create their own regressors. Note, undifferencing the series deletes all added regressors.

In [ ]:
f.undiff()
f

In [ ]:
f.set_estimator('prophet')
f.manual_forecast()

In [ ]:
f.set_estimator('silverkite')
f.manual_forecast()
f.save_summary_stats()

Note, summary stats are available for silverkite but not Prophet -- contributions are welcome to get summary stats for Prophet.

After forecasting with Silverkite, we need to reset some matplotlib parameters for plotting to work.

In [ ]:
matplotlib.use('nbAgg')
%matplotlib inline
sns.set(rc={'figure.figsize':(15,7)})

Let's see how these 2 new models performed compared with the top-3 models identified before.

In [ ]:
f.plot_test_set(models=['hwes_seasonal','arima_ma_terms','mlr','prophet','silverkite'],order_by='LevelTestSetRMSE',level=True)

And their trends into the future:

In [ ]:
f.plot(models=['hwes_seasonal','arima_ma_terms','mlr','prophet','silverkite'],order_by='LevelTestSetRMSE',print_attr=['LevelTestSetRMSE'],level=True)

Neither the prophet or silverkite models proved to be as accurate as the HWES or ARIMA models, but the Prophet did outperform MLR.

### TensorFlow Recurrent Neural Nets
[Back to top](#eCommerce-Example)  

The most advanced models available in scalecast are recurrent neural networks. There are two kinds: the SimpleRNN and the Long Short-Term Memory. The SimpleRNN, perhaps ironically, has the more complicated implementation in this framework. It allows for nearly all customization of the model that TensorFlow itselt would offer. What scalecast will do for you is preprocess the data correctly, scale all of the data when training the model, and unscale it appropriately to compare to any other models you run. It also tests the model on the same test set and bootsraps confidence intervals. The fact that it tests all the models also means it fits all the models twice, which can slow you down sometimes, but having two fits is also a blessing: **it really lets you know how generalizable your model is when new data is added.** Let's explore the SimpleRNN more carefully.

#### SimpleRNN
[Back to top](#eCommerce-Example)  

See the [documentation](https://scalecast.readthedocs.io/en/latest/Forecaster/_forecast.html#module-src.scalecast.Forecaster.Forecaster._forecast_rnn) for the 'rnn' estimator. In practice, calling this model looks like:

In [ ]:
f.set_estimator('rnn')
f.manual_forecast(lags=30,
                  hidden_layers_struct={'simple':{'units':64},'simple':{'units':64},'simple':{'units':64}},
                  validation_split=0.2,
                  epochs=15,
                  plot_loss=True,
                  call_me='simple_rnn')

f.manual_forecast(lags=30,
                  hidden_layers_struct={'lstm':{'units':64},'lstm':{'units':64},'lstm':{'units':64}},
                  validation_split=0.2,
                  epochs=15,
                  plot_loss=True,
                  call_me='lstm')

Anything the lstm can do in scalecast, the rnn can also do. Using the lstm estimator instead of rnn is simpler, and that is explored below.

#### LSTM
[Back to top](#eCommerce-Example)  

See the [documentation](https://scalecast.readthedocs.io/en/latest/Forecaster/_forecast.html#module-src.scalecast.Forecaster.Forecaster._forecast_lstm) for the 'lstm' estimator. There are more parameters to explicitly specify, but that makes it a lot easier to navigate as well. In practice, calling this model looks like:

In [ ]:
f.set_estimator('lstm')
f.manual_forecast(lags=30,
                  lstm_layer_sizes=(64,64,64),
                  dropout=(0.2,0,0),
                  validation_split=0.2,
                  epochs=15,
                  plot_loss=True,
                  call_me='lstm_regularized')

Finally, we can see how all these models performed on the test set:

In [ ]:
f.plot_test_set(models=['simple_rnn','lstm','lstm_regularized'],ci=True,order_by='LevelTestSetRMSE')

And into the future:

In [ ]:
f.plot(models=['simple_rnn','lstm','lstm_regularized'],ci=True,order_by='LevelTestSetRMSE',level=True,print_attr=['LevelTestSetRMSE'])

None of these models arrived at the level of the MLR, ARIMA, or HWES models. That's not to say they couldn't if more time weren't spent with them. Theoretically, they can specify thousands of parameters and find the true long-term trends in the data that none of the other models would be capable of. For some datasets, this makes a big difference. But some datasets are too random to be fit effectively by one of these more advanced models. Sometimes the simplest is most accurate, fastest, and most interpretable. That being said, all of these models probably could have benefited from more training epochs.

### Combination Modeling

[Back to top](#eCommerce-Example)  
The last model concept we will explore is combination modeling. There are three types of combinations:
- simple: a simple average of a group of models.
- weighted: a weighted average of a group of models. Weights can be passed manually (as we do below) or set automatically based on a metric passed to the `determine_best_by` parameter.
- splice: a splice of two or more models at one or more future splice points. All metrics, fitted values, and test-set metrics from this model will be identical to the simple average.

We will be using simple and weighted average combination modeling only.

In [ ]:
f.set_estimator('combo')
f.manual_forecast(how='simple',models=['hwes_seasonal','arima_ma_terms','prophet','silverkite','simple_rnn','lstm','lstm_regularized'],call_me='avg_lvl_models')
f.manual_forecast(how='weighted',models=_sklearn_estimators_,determine_best_by='ValidationMetricValue',call_me='weighted_differenced_models')

It could be argued that combination modeling is prone to problems related to data leakage--we shouldn't select model combinations based based on their performance on the test set and then recompare them to the test set. By default, deterimine_best_by in both of these combination types is `'ValidationMetricValue'`. This is a way to ensure data leakage does not occur. Since we didn't tune all the models we want to combine in the first function, we simply write out the models we want to combine in a list.

The weighted average model can accept weights as arguments that add to 0 or not--if not, they will be rebalanced to do so. If determine_best_by is specified, weights can be left to None and the weights will be chosen automatically. Again, be careful to not overfit models this way.

All information available for other models, including fitted values and test-set metrics, are also available for these combo models.

In [ ]:
f.plot_test_set(models='top_5',order_by='LevelTestSetRMSE',level=True)

Again, we see the usual suspects. One more time on future data:

In [ ]:
f.plot(models='top_5',order_by='LevelTestSetRMSE',print_attr=['LevelTestSetRMSE'],level=True)

And just as a grand finale, let's see all evaluated models plotted together:

In [ ]:
f.plot(order_by='LevelTestSetRMSE',print_attr=['LevelTestSetRMSE'],level=True)

## Export Results

[Back to top](#eCommerce-Example)  
Now that we have models that offer interesting ranges of predictions (and a few that look pretty bad), let's export all available results to Excel to view later. The first function below exports a workbook with five tabs that offer information including forecasted values, level forecasted values, descriptive information about each model, and test-set prediction. The second two functions export feature info and validation grids, and they have to_excel in the method name because they automatically put each model's info on a seperate tab.

In [ ]:
results = f.export(to_excel=True,determine_best_by='LevelTestSetRMSE',excel_name='eCommerce_results.xlsx')
f.all_feature_info_to_excel(excel_name='eCommerce_feature_info.xlsx') 
f.all_validation_grids_to_excel(sort_by_metric_value=True,excel_name='eCommerce_validation_grids.xlsx')

One of the most interesting of the exports above is the model summaries dataframe. It has many columns that you can view in the exported Excel workbook, but a sample of this dataframe is displayed below.

In [ ]:
results['model_summaries'][['ModelNickname','Estimator','HyperParams','LevelTestSetRMSE','LevelTestSetR2']]

Here is a list of all export functions. Most of these output results to a pandas dataframe.

In [ ]:
print(*f.get_funcs('exporter'),sep='\n')